<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/10_event_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

10.　イベント・スタディ法による株価効果の分析

　証券アナリストや学術研究の世界において、企業のM&Aを分析する際に最も多用されている分析手法が「イベント・スタディ」です。企業の財務情報ではなく、株価を用いた短期効果が計測される理由は、３カ月に一度の財務情報の場合、M&A以外の影響が財務パフォーマンスに反映されることが多いためです。以下では、2018年5月8日に発表された武田薬品工業(4502.JP)による英シャイアのM&A株価効果のイベント・スタディを事例として紹介します。

In [1]:
#[1]ライブラリ読み込み
import pandas as pd
import pandas_datareader as data
import matplotlib.pyplot as plt
import numpy as np
import datetime
import statsmodels.api as sm

　武田薬品工業と日経平均株価の証券コードを指定し、2-3行目では標本期間を指定します。重要なポイントは、シングル・ファクター・モデルを用いて企業の株価理論値を計測している点です。Brown and Warner (1985)にしたがい、M&Aアナウンス日より250営業日までの過去の日次データから直近1週間のデータを取り除いたうえで、次式を推計し、理論値を算出します。

In [2]:
#[2]市場インデックスと買収企業株価の取得
tickers=['4502.JP','^NKX']#買収企業の証券コードと市場インデクス株価のコードを記述
start = datetime.date(2017,4,8)#買収発表日から250営業日前の日次データ開始日を設定
end = datetime.date(2018,5,15)#買収発表日から7日後の日次データ最終日を設定
df=data.DataReader(tickers,'stooq',start=start,end=end).sort_values(by='Date',ascending=True)

　累積超過収益率（CAR）の算出では、取得した日次データを分割するため、日付情報のデータ形式を確認することが必要となります。取得した株価データの変動率を算出した上で、日付情報を確認するため、データ型を確認します。本例のstooqから取得した株価データの形式はdatetime64[ns]であるため、これを変換せずにそのままデータとして用いることとします。

In [3]:
#[3]データ前処理
df=df.Close.pct_change().dropna()
company_list=['Takeda','Nikkei225']
df.columns = company_list
df.reset_index(inplace=True)#変数名リセット
df.dtypes#データ型の確認

Date         datetime64[ns]
Takeda              float64
Nikkei225           float64
dtype: object

　次に、取得した日次データを用いて、理論値推計のための標本、3種類の累積超過収益率（CAR）算出のための標本期間、計4種類の標本を作成します。イベント・ウィンドウはこれら以外にも（-1，+10）や（-5，+5）など、分析と研究の問題意識に応じて、様々な期間設定を行うことができます。ただし、イベント・ウィンドウの期間が長すぎると、その間、イベント以外のノイズが株価に影響をもたらす可能性が高まることに留意してください。尚、本例ではquery()を用いて、標本期間を指定していますが、データ形式がdatetime64[ns]である場合、iloc[]やdt.datetime()を用いても構いません。

In [4]:
#[4]株価標本データの分割
df1=df.query('"2017-05-01" < Date < "2018-04-30"')#理論値推計の標本期間
CAR3=df.query('"2018-05-06" < Date < "2018-05-10"')#CAR(-1,+1)の標本期間
CAR5=df.query('"2018-05-06" < Date < "2018-05-13"')#CAR(-1,+3)の標本期間
CAR7=df.query('"2018-05-06" < Date < "2018-05-16"')#CAR(-1,+5)の標本期間

　次に4種類の標本期間のうちの最初のデータを用いて、statsmodelsによりシングル・ファクター・モデルの推計を行います。下記の推計結果の係数値は、この標本期間の武田薬品工業の株価ベータ値を示しています。

In [ ]:
#[5]企業株価シングル・ファクター・モデルの推計
X=df1[['Nikkei225']]
y=df1['Takeda']
X=sm.add_constant(X)
Model=sm.OLS(y,X).fit()
print(Model.summary().tables[1])
print('AdjR2: %.4f'% Model.rsquared_adj)

　推計されたベータ値から企業の株価理論値を算出し、最後にこれを用いて本例では3種類のイベント・ウィンドウによる累積超過収益率（CAR）を算出します。

In [ ]:
#[6]累積超過収益率(CAR）の算出
a = Model.params[1]#シングル・ファクター・モデルの係数値の定義
b = Model.params[0]#シングル・ファクター・モデルの定数項の定義
CAR3['predict3']=CAR3[['Nikkei225']]*a+b#理論値の算出
CAR3['actual3']=CAR3[['Takeda']]#実績値の定義
CAR3['difference']=CAR3['actual3']-CAR3['predict3']#実績値と理論値の差分

CAR5['predict5']=CAR5[['Nikkei225']]*a+b
CAR5['actual5']=CAR5[['Takeda']]
CAR5['difference']=CAR5['actual5']-CAR5['predict5']

CAR7['predict7']=CAR7[['Nikkei225']]*a+b
CAR7['actual7']=CAR7[['Takeda']]
CAR7['difference']=CAR7['actual7']-CAR7['predict7']

print('CAR(-1,+1): %.4f'% CAR3['difference'].sum(),'CAR(-1,+3): %.4f'% CAR5['difference'].sum(),'CAR(-1,+5): %.4f'% CAR7['difference'].sum())